In [20]:
import numpy as np
import pandas as pd

In [28]:
forecasts = pd.read_csv("prediction_datasets/two_room/two_room_sarimax_forecasts.csv", index_col=[0]).T
forecasts.reset_index(inplace=True)
forecasts.rename(columns={"index": "Postal code"}, inplace=True)
forecasts.head()

,Postal code,pred_0,pred_1,pred_2,pred_3
0,00100,1.376730e+06,1.440879e+06,1.519348e+06,1.588185e+06
1,00120,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00
2,00140,9.067857e+03,9.762777e+03,9.846263e+03,9.351347e+03
3,00150,8.903481e+03,8.810338e+03,8.887704e+03,9.195685e+03
4,00170,8.226921e+03,8.011609e+03,8.317305e+03,8.341036e+03


In [29]:
import pgeocode

nomi = pgeocode.Nominatim('fi')
location = nomi.query_postal_code(forecasts["Postal code"].values) # df_p_col = postal codes
wanted_features_location = location[["postal_code",
                                     "latitude", 
                                     "longitude"]]
wanted_features_location = wanted_features_location.rename(columns={"postal_code": "Postal code"})
new_json_df = pd.merge(forecasts, wanted_features_location, on="Postal code").set_index("Postal code") #json_df = predictions
new_json_df.head()

,pred_0,pred_1,pred_2,pred_3,latitude,longitude
Postal code,,,,,,
00100,1.376730e+06,1.440879e+06,1.519348e+06,1.588185e+06,60.1714,24.9316
00120,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,60.1632,24.9391
00140,9.067857e+03,9.762777e+03,9.846263e+03,9.351347e+03,60.1578,24.9525
00150,8.903481e+03,8.810338e+03,8.887704e+03,9.195685e+03,60.1570,24.9369
00170,8.226921e+03,8.011609e+03,8.317305e+03,8.341036e+03,60.1728,24.9554


In [30]:
new_json_df.to_json("prediction_datasets/two_room/two_room_sarimax_forecasts.json")

# Data Group A

In [65]:
dfA = pd.read_csv("./imputed_df.csv",index_col=[0])
dfA.head()

,"(100, 'Blocks of flats, one-room flat')","(100, 'Blocks of flats, two-room flat')","(100, 'Blocks of flats, three-room flat+')","(100, 'Blocks of flats total')","(100, 'Building types total')","(120, 'Blocks of flats, one-room flat')","(120, 'Blocks of flats, two-room flat')","(120, 'Blocks of flats, three-room flat+')","(120, 'Blocks of flats total')","(120, 'Building types total')",...,"(96200, 'Building types total')","(96300, 'Blocks of flats, two-room flat')","(96300, 'Building types total')","(96400, 'Building types total')","(96440, 'Building types total')","(96460, 'Building types total')","(96500, 'Building types total')","(96900, 'Building types total')","(96910, 'Building types total')","(99600, 'Building types total')"
2010-04-01,5347,5021,5396,5219,5219,5646,5355,5713,5495,5495,...,1646,1525,1475,1499,1109,1568,1039,2030,1404,1059
2010-07-01,5826,5081,4828,5181,5181,5395,5534,5884,5595,5595,...,1668,1794,1782,1614,1069,1568,1069,1519,1354,836
2010-10-01,5566,5006,5394,5269,5269,5613,5321,5666,5521,5521,...,1619,1670,1750,1826,929,1568,1028,1839,1462,972
2011-01-01,5545,5713,5571,5632,5632,5905,5528,5691,5689,5689,...,1631,1501,1678,1697,1195,1615,1146,1868,1445,737
2011-04-01,5812,5866,5709,5807,5807,6332,5523,5571,5749,5749,...,1906,1713,1672,1565,932,1657,1097,1729,1544,932


In [149]:
dfA.info()

<class 'pandas.core.frame.DataFrame'>
Index: 45 entries, 2010-04-01 to 2021-04-01
Columns: 1193 entries, (100, 'Blocks of flats, one-room flat') to (99600, 'Building types total')
dtypes: int64(1193)
memory usage: 419.8+ KB


Unpacking tuple strings into `zip_codes` and `building_types`

In [66]:
import re

def to_zipcode(tuple_string):
    return f"{int(re.search(r'[0-9]+', tuple_string).group()):05d}"

def to_building_type(tuple_string):
    return tuple_string.split("'")[1]
    

print(to_zipcode(dfA.columns[0]))
print(to_building_type(dfA.columns[0]))

00100
Blocks of flats, one-room flat


In [67]:
zip_codes, building_types = [to_zipcode(col) for col in dfA.columns], [to_building_type(col) for col in dfA.columns]
zip_codes[:5],building_types[:5]

(['00100', '00100', '00100', '00100', '00100'],
 ['Blocks of flats, one-room flat',
  'Blocks of flats, two-room flat',
  'Blocks of flats, three-room flat+',
  'Blocks of flats total',
  'Building types total'])

# Data Group B

In [119]:
dfB = pd.read_csv("cleaned.csv")
dfB.head()

,Postal code,Building type,2010Q1 Price per square meter (EUR/m2),2010Q2 Price per square meter (EUR/m2),2010Q3 Price per square meter (EUR/m2),2010Q4 Price per square meter (EUR/m2),2011Q1 Price per square meter (EUR/m2),2011Q2 Price per square meter (EUR/m2),2011Q3 Price per square meter (EUR/m2),2011Q4 Price per square meter (EUR/m2),...,2019Q1 Price per square meter (EUR/m2),2019Q2 Price per square meter (EUR/m2),2019Q3 Price per square meter (EUR/m2),2019Q4 Price per square meter (EUR/m2),2020Q1 Price per square meter (EUR/m2),2020Q2 Price per square meter (EUR/m2),2020Q3 Price per square meter (EUR/m2),2020Q4 Price per square meter (EUR/m2),2021Q1* Price per square meter (EUR/m2),2021Q2* Price per square meter (EUR/m2)
0,100,"Blocks of flats, one-room flat",5458.0,5347.0,5826.0,5566.0,5545.0,5812.0,5909.0,5856.0,...,7755.0,8332.0,8130.0,8068.0,7802.0,8543.0,9170.0,7517.0,8872.0,8596.0
1,100,"Blocks of flats, two-room flat",5164.0,5021.0,5081.0,5006.0,5713.0,5866.0,5552.0,5614.0,...,6914.0,7694.0,6901.0,7435.0,7573.0,7398.0,7868.0,7700.0,7829.0,7974.0
2,100,"Blocks of flats, three-room flat+",4944.0,5396.0,4828.0,5394.0,5571.0,5709.0,5450.0,5010.0,...,6883.0,6846.0,7379.0,6728.0,7205.0,6678.0,7178.0,7405.0,8053.0,8174.0
3,100,Blocks of flats total,5168.0,5219.0,5181.0,5269.0,5632.0,5807.0,5630.0,5511.0,...,7043.0,7478.0,7476.0,7345.0,7460.0,7275.0,8058.0,7540.0,8188.0,8188.0
4,100,Terraced houses total,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [120]:
### Regex to change quarters to datetime
import datetime as date

def quarter_to_datetime(row):
    dates = {"Q1": "1/1", "Q2":"1/4", "Q3":"1/7", "Q4":"1/10"}
    string = row.replace("*", "").split(" ")[0]
    year, quarter = string[:4], string[4:]
    datetime_str = dates[quarter] + "/" + year
    return date.datetime.strptime(datetime_str, '%d/%m/%Y')

In [121]:
indices = list(zip(dfB.iloc[:,0].values, dfB.iloc[:,1].values))
columns = dfB.iloc[:,2:].columns.to_series().apply(quarter_to_datetime).values

print(indices[:5])
print(columns[:5])

[(100, 'Blocks of flats, one-room flat'), (100, 'Blocks of flats, two-room flat'), (100, 'Blocks of flats, three-room flat+'), (100, 'Blocks of flats total'), (100, 'Terraced houses total')]
['2010-01-01T00:00:00.000000000' '2010-04-01T00:00:00.000000000'
 '2010-07-01T00:00:00.000000000' '2010-10-01T00:00:00.000000000'
 '2011-01-01T00:00:00.000000000']


In [122]:
dfB = pd.DataFrame(dfB.iloc[:,2:].to_numpy(), index=indices, columns=columns).T

dfB.head()

,"(100, Blocks of flats, one-room flat)","(100, Blocks of flats, two-room flat)","(100, Blocks of flats, three-room flat+)","(100, Blocks of flats total)","(100, Terraced houses total)","(100, Building types total)","(120, Blocks of flats, one-room flat)","(120, Blocks of flats, two-room flat)","(120, Blocks of flats, three-room flat+)","(120, Blocks of flats total)",...,"(99950, Blocks of flats, three-room flat+)","(99950, Blocks of flats total)","(99950, Terraced houses total)","(99950, Building types total)","(99980, Blocks of flats, one-room flat)","(99980, Blocks of flats, two-room flat)","(99980, Blocks of flats, three-room flat+)","(99980, Blocks of flats total)","(99980, Terraced houses total)","(99980, Building types total)"
2010-01-01,5458.0,5164.0,4944.0,5168.0,NaN,5168.0,5515.0,5349.0,6236.0,5618.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2010-04-01,5347.0,5021.0,5396.0,5219.0,NaN,5219.0,5646.0,5355.0,5713.0,5495.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2010-07-01,5826.0,5081.0,4828.0,5181.0,NaN,5181.0,5395.0,5534.0,5884.0,5595.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2010-10-01,5566.0,5006.0,5394.0,5269.0,NaN,5269.0,5613.0,5321.0,5666.0,5521.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2011-01-01,5545.0,5713.0,5571.0,5632.0,NaN,5632.0,5905.0,5528.0,5691.0,5689.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [ ]:
dfA

In [145]:
nice_columns = dfB.columns[dfB.isna().sum() <= 46/100*40]
nice = dfB[nice_columns]
nice.head()

,"(100, Blocks of flats, one-room flat)","(100, Blocks of flats, two-room flat)","(100, Blocks of flats, three-room flat+)","(100, Blocks of flats total)","(100, Building types total)","(120, Blocks of flats, one-room flat)","(120, Blocks of flats, two-room flat)","(120, Blocks of flats, three-room flat+)","(120, Blocks of flats total)","(120, Building types total)",...,"(96200, Building types total)","(96300, Blocks of flats, two-room flat)","(96300, Building types total)","(96400, Building types total)","(96440, Building types total)","(96460, Building types total)","(96500, Building types total)","(96900, Building types total)","(96910, Building types total)","(99600, Building types total)"
2010-01-01,5458.0,5164.0,4944.0,5168.0,5168.0,5515.0,5349.0,6236.0,5618.0,5618.0,...,1672.0,NaN,1758.0,1539.0,896.0,1598.0,970.0,1837.0,1292.0,830.0
2010-04-01,5347.0,5021.0,5396.0,5219.0,5219.0,5646.0,5355.0,5713.0,5495.0,5495.0,...,1646.0,1525.0,1475.0,1499.0,1109.0,1568.0,1039.0,2030.0,1404.0,NaN
2010-07-01,5826.0,5081.0,4828.0,5181.0,5181.0,5395.0,5534.0,5884.0,5595.0,5595.0,...,1668.0,1794.0,1782.0,1614.0,1069.0,NaN,1069.0,1519.0,1354.0,836.0
2010-10-01,5566.0,5006.0,5394.0,5269.0,5269.0,5613.0,5321.0,5666.0,5521.0,5521.0,...,1619.0,NaN,1750.0,1826.0,929.0,NaN,1028.0,NaN,1462.0,972.0
2011-01-01,5545.0,5713.0,5571.0,5632.0,5632.0,5905.0,5528.0,5691.0,5689.0,5689.0,...,1631.0,1501.0,1678.0,1697.0,1195.0,1615.0,1146.0,1868.0,1445.0,737.0


In [148]:
dfA

,"(100, 'Blocks of flats, one-room flat')","(100, 'Blocks of flats, two-room flat')","(100, 'Blocks of flats, three-room flat+')","(100, 'Blocks of flats total')","(100, 'Building types total')","(120, 'Blocks of flats, one-room flat')","(120, 'Blocks of flats, two-room flat')","(120, 'Blocks of flats, three-room flat+')","(120, 'Blocks of flats total')","(120, 'Building types total')",...,"(96200, 'Building types total')","(96300, 'Blocks of flats, two-room flat')","(96300, 'Building types total')","(96400, 'Building types total')","(96440, 'Building types total')","(96460, 'Building types total')","(96500, 'Building types total')","(96900, 'Building types total')","(96910, 'Building types total')","(99600, 'Building types total')"
2010-04-01,5347,5021,5396,5219,5219,5646,5355,5713,5495,5495,...,1646,1525,1475,1499,1109,1568,1039,2030,1404,1059
2010-07-01,5826,5081,4828,5181,5181,5395,5534,5884,5595,5595,...,1668,1794,1782,1614,1069,1568,1069,1519,1354,836
2010-10-01,5566,5006,5394,5269,5269,5613,5321,5666,5521,5521,...,1619,1670,1750,1826,929,1568,1028,1839,1462,972
2011-01-01,5545,5713,5571,5632,5632,5905,5528,5691,5689,5689,...,1631,1501,1678,1697,1195,1615,1146,1868,1445,737
2011-04-01,5812,5866,5709,5807,5807,6332,5523,5571,5749,5749,...,1906,1713,1672,1565,932,1657,1097,1729,1544,932
2011-07-01,5909,5552,5450,5630,5630,6034,5594,5512,5518,5518,...,1855,1620,1698,1680,1090,1694,1008,2035,1454,1082
2011-10-01,5856,5614,5010,5511,5511,5988,5045,5317,5471,5471,...,1856,1709,1813,1354,1117,1609,1095,1971,1467,1241
2012-01-01,5785,6020,5704,5843,5843,5824,5870,5858,5851,5851,...,1756,1672,1792,1709,1199,1559,1026,1953,1524,1129
2012-04-01,6436,5814,5466,5899,5899,5890,6080,5800,5957,5957,...,1814,1761,1708,1722,1115,1619,1162,1879,1552,1186
2012-07-01,6265,5802,5778,5965,5965,6503,5652,5984,5987,5987,...,1632,1839,1804,1917,1244,1607,1055,1620,1434,1198


In [131]:
df3 = pd.merge(dfA, dfB, how='outer', indicator='Exist')
df3 = df3.loc[df3['Exist'] != 'both']


MergeError: No common columns to perform merge on. Merge options: left_on=None, right_on=None, left_index=False, right_index=False

Random

In [1]:
pd.read_csv("notebooks/Bruce/prediction_datasets/one_room/one_room_sarimax_forecasts.csv")

NameError: name 'pd' is not defined